# <b> Project 1

<b>Introduction<br></b>
<p>In this project, there will be three parts to analyze pandas. First part will be Grammar for pandas library. Second part will implement an algorithm to identify if the input is vaild DataFrame structure. Third part will be the cost analysis of DataFrame and type of the parameter . 

<div style="background-color:#E1F8E1; padding:5px 8px 5px 8px;">
The following BNF grammar definition of Pandas:

\begin{eqnarray*}
\textit{ mod}& ::= &\textbf{Module(stmt* body))} \\
                 &  |  &\textbf{Expression(expr body)} \\
 & & \\
\textit{stmt}& ::= &\textbf{Assign(expr* targets, stmt value))} \\
 & & \\
 & & \\
\textit{expr}& ::= &\textbf{Attribute(expr value, identifier attr, expr_context ctx)} \\
                 &  |  &\textbf{ Num(object n)} \\
                 &  |  &\textbf{ Subscript(expr value, slice slice, expr_context ctx)} \\
                 &  |  &\textbf{ Str(string s)} \\
                 &  |  &\textbf{ Name(identifier id, expr_context ctx)} \\
                 &  |  &\textbf{ Dict(expr* keys,  expr* values)} \\
                 &  |  &\textbf{List(expr* elts, expr_context ctx)} \\
                 &  |  &\textbf{Tuple(expr* elts, expr_context ctx)} \\
                 &  |  &\textbf{Slice(expr* value)} \\
                 &  |  &\textbf{Call(expr func, expr* args, keyword* keywords)} \\
                 &  |  &\textbf{BinOp(expr left, operator op, expr right)} \\
 & & \\
\textit{operator}& ::= &\textbf{Add } \\
 & & \\
  & & \\
\text{keyword arguments supplied to call:}\\
\textit{keyword}& ::= &\textbf{(identifier? arg, expr value) } \\
\end{eqnarray*}
</div>

<b>Part II</b>
<p>Below is an implementation of an algorithm that determine if the input statement can be transform or is already DataFrame or not
<br>

In [6]:
#This 
import ast 
import pandas as pd
class is_DataFrame(ast.NodeVisitor):
    def __init__(self):
        '''a parameter to check when pass in a dictionary, there must be a index'''
        self.dic = False
        
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        return all(results)

    def visit_Assign(self, node):
        return self.visit(node.value)
    
    def visit_Expr(self, node):
        return self.visit(node.value)
    
    def visit_Call(self, node):
        if len(node.keywords) > 1:
            for each in node.keywords:
                if each.arg == 'index':
                    self.dic = True
        a = node.keywords[0]
        results = self.visit(a)
        return results
    
    def visit_keyword(self, node):
        return self.visit(node.value)
    
    def visit_BinOp(self, node):
        return self.visit(node.left) and self.visit(node.right)

    def visit_List(self, node):
        return True
    
    def visit_Dict(self, node):
        '''need to specify the if index exist'''
        return self.dic
    
    def visit_Subscript(self, node):
        return True
    


test1 = ast.parse('pd.DataFrame(data = [1,2,3] ) + pd.DataFrame(data = [1,2,3] )')
test2 = ast.parse('pd.DataFrame(data = {1:1,2:2},index = {1,2})')
test3 = ast.parse('k = pd.DataFrame(data = [1,2,3] )')
test4 = ast.parse('pd.DataFrame(data = {1,2,3} )')
print("Test1: ", is_DataFrame().visit(test1))
print("Test2: ", is_DataFrame().visit(test2))
print("Test3: ", is_DataFrame().visit(test3))
print("Test4: ", is_DataFrame().visit(test4))

Test1:  True
Test2:  True
Test3:  True
Test4:  False


<b>Part III</b>
<p>a) Implementation of a static analysis algorithm that infers the type of a single DataFrame expression.

In [8]:
import ast 
import pandas as pd

class infer_type(ast.NodeVisitor):
    
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        if results == [dict] or results == [list]:
            return results[0]
        
    def visit_Assign(self, node):
        return self.visit(node.value)

    def visit_Expr(self, node):
        return self.visit(node.value)

    def visit_Call(self, node):
        a = node.keywords[0]
        results = self.visit(a)
        return results
    
    def visit_keyword(self, node):
        return self.visit(node.value)

    def visit_BinOp(self, node):
        if type(node.op) in [ast.Add]:
            if self.visit(node.left) is list and\
               self.visit(node.right) is list:
                return list    
    
    def visit_List(self, node):
        return list
    
    def visit_Dict(self, node):
        return dict


In [20]:
def typeInput(newInput):
    '''test your input if is a dataframe then try to 
        find out the type of you dataframe parameter'''
    test = ast.parse(newInput)
    res = is_DataFrame().visit(test)
    if res == True:
        return infer_type().visit(test)
    else:
        return "Your input is not a valid DataFrame input"
    

type_test1 = 'k = pd.DataFrame(data = [1,2,3] )'
type_test2 = 'pd.DataFrame(data = {1:1,2:2},index = {1,2})'
type_test3 = 'pd.DataFrame(data = 1 )'
type_test4 = 'pd.DataFrame(data = {1,2,3} )'
type_test5 = 'pd.DataFrame(data = {1:"ert",2:"wier"},index = {1,2})'
print("Type Test1: ", typeInput(type_test1))
print("Type Test2: ", typeInput(type_test2))
print("Type Test3: ", typeInput(type_test3))
print("Type Test4: ", typeInput(type_test4))
print("Type Test5: ", typeInput(type_test5))

Type Test1:  <class 'list'>
Type Test2:  <class 'dict'>
Type Test3:  Your input is not a valid DataFrame input
Type Test4:  Your input is not a valid DataFrame input
Type Test5:  <class 'dict'>


<p>b) One dimensional DataFrame Cost analysis without runing code

In [16]:
class arr_memory_check(ast.NodeVisitor):
    
    def __init__(self):
        self.nn = 0
        self.single = 1
    
    def visit_Module(self, node):
        results = [self.visit(s) for s in node.body]
        return results[0]
    
    def visit_Assign(self, node):
        return self.visit(node.value)

    def visit_Expr(self, node):
        return self.visit(node.value)

    def visit_Call(self, node):
        results = [self.visit(s) for s in node.keywords]
        return results[0]
    
    def visit_keyword(self, node):
        return self.visit(node.value)

    def visit_BinOp(self, node):
         return self.visit(node.left) + self.visit(node.right)    
    
    def visit_List(self, node):
        self.nn = len(node.elts)
        result = [self.visit(s) for s in node.elts]
        return result[0]
    
    def visit_Dict(self, node):
        self.nn = len(node.keys)
        result = [self.visit(s) for s in node.values]
        return result[0]
    
    def visit_Num(self, node):
        self.single = 8
        return self.nn * self.single
    
    def visit_Str(self, node):
        t = len(node.s)
        self.single = 8*t
        return self.nn * self.single


In [18]:
def cost(newInput):
    '''test your input if is a dataframe then try to 
        find out the memory that your parameter will take
        without executing the code'''
    test = ast.parse(newInput)
    res = is_DataFrame().visit(test)
    if res == True:
        return arr_memory_check().visit(test)
    else:
        return "Your input is not a valid DataFrame input"

cost_test1 = 'pd.DataFrame(data = [1,2,3] )'
cost_test2 = 'pd.DataFrame(data = {1:1,2:2},index = {1,2})'
cost_test3 = 'pd.DataFrame(data = 1 )'
cost_test4 = 'pd.DataFrame(data = {1,2,3} )'
cost_test5 = 'k = pd.DataFrame(data = [1,2,3]+[4] )'
cost_test6 = 'pd.DataFrame(data = {1:"ert",2:"wier"},index = {1,2})'
print("Cost Test1 (bits): ", cost(cost_test1))
print("Cost Test2 (bits): ", cost(cost_test2))
print("Cost Test3 (bits): ", cost(cost_test3))
print("Cost Test4 (bits): ", cost(cost_test4))
print("Cost Test5 (bits): ", cost(cost_test5))
print("Cost Test6 (bits): ", cost(cost_test6))

Cost Test1 (bits):  24
Cost Test2 (bits):  16
Cost Test3 (bits):  Your input is not a valid DataFrame input
Cost Test4 (bits):  Your input is not a valid DataFrame input
Cost Test5 (bits):  32
Cost Test6 (bits):  48
